In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df1=pd.read_csv('/kaggle/input/tuberculosis-chest-xrays-shenzhen/shenzhen_metadata.csv')
base_path='/kaggle/input/tuberculosis-chest-xrays-shenzhen/images/images/'
df1['equalized_filename'] = df1['study_id'].apply(lambda x: base_path+x)
df2=pd.read_csv('/kaggle/input/tuberculosis-chest-xrays-montgomery/montgomery_metadata.csv')
base_path='/kaggle/input/tuberculosis-chest-xrays-montgomery/images/images/'
df2['equalized_filename'] = df2['study_id'].apply(lambda x: base_path+x)
df=pd.concat([df1, df2], ignore_index=True)
df['target']=df['findings'].apply(lambda x:0 if x=='normal' else 1)
df['target'] = df['target'].astype(str)
df = df.sample(frac=1).reset_index(drop=True)

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-04-23 05:28:49.546621: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 05:28:49.546718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 05:28:49.663002: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Split the data into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['target'], random_state=42)

# Set up data generators
batch_size =16
image_size = (512, 512)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='equalized_filename',
    y_col='target',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',  # Set class_mode to 'binary'
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_dataframe(
    val_df,
    x_col='equalized_filename',
    y_col='target',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',  # Set class_mode to 'binary'
    subset='validation'
)

Found 512 validated image filenames belonging to 2 classes.
Found 32 validated image filenames belonging to 2 classes.


In [5]:
!pip install efficientnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 694.2 kB/s eta 0:00:00--:--


In [7]:
import efficientnet.tfkeras as efn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Input

from tensorflow.keras.optimizers import Adam


base_model_e1 = efn.EfficientNetB1(weights='imagenet', include_top=False, input_shape=(512, 512, 3))

# Optionally freeze the layers of the base model
for layer in base_model_e1.layers:
    layer.trainable = False

# Add custom classification head
model_e1 = Sequential([
    Input(shape=(512, 512, 3)),
    base_model_e1,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')  # Change the number of units according to your task
])


# Define a custom learning rate for the Adam optimizer
custom_learning_rate = 0.00005  # Adjust the learning rate as needed

# Create an instance of the Adam optimizer with the custom learning rate
optimizer = Adam(learning_rate=custom_learning_rate)

# Compile the model
model_e1.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model_e1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnet-b1 (Functional)    │ (None, 16, 16, 1280)   │     6,575,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,903,425 (26.33 MB)

 Trainable params: 328,193 (1.25 MB)

 Non-trainable params: 6,575,232 (25.08 MB)

In [8]:
history_efficientNet = model_e1.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator)
)


Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 1/32 ━━━━━━━━━━━━━━━━━━━━ 44:29 86s/step - accuracy: 0.5625 - loss: 0.7412

I0000 00:00:1713850321.660630      92 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1713850321.728285      92 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4817 - loss: 0.7116

W0000 00:00:1713850419.098787      90 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


32/32 ━━━━━━━━━━━━━━━━━━━━ 184s 3s/step - accuracy: 0.4836 - loss: 0.7108 - val_accuracy: 0.6250 - val_loss: 0.6494
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/50


/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


32/32 ━━━━━━━━━━━━━━━━━━━━ 90s 2s/step - accuracy: 0.7300 - loss: 0.6235 - val_accuracy: 0.7188 - val_loss: 0.5823
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 725us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - accuracy: 0.7598 - loss: 0.5685 - val_accuracy: 0.7500 - val_loss: 0.5497
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 96s 2s/step - accuracy: 0.7058 - loss: 0.5734 - val_accuracy: 0.8125 - val_loss: 0.5231
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 736us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 90s 2s/step - accuracy: 0.7174 - loss: 0.5580 - val_accuracy: 0.7812 - val_loss: 0.4855
Epoch 10/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 604us/step - accuracy: 0.0000e+00